In [7]:
import cv2
import torch
import os
import numpy as np
import os
from collections import defaultdict


In [8]:
!pip install deep-sort-realtime ultralytics opencv-python numpy




In [9]:
from deep_sort_realtime.deepsort_tracker import DeepSort
tracker = DeepSort(max_age=30)
print("DeepSORT installed and working!")


DeepSORT installed and working!


In [10]:
!pip install ultralytics

In [11]:
from ultralytics import YOLO

In [12]:
from ultralytics import YOLO

# Download YOLOv9 model weights
!wget https://github.com/SkalskiP/yolov9/releases/download/v1.0/yolov9.pt




--2025-02-06 04:39:43--  https://github.com/SkalskiP/yolov9/releases/download/v1.0/yolov9.pt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-06 04:39:43 ERROR 404: Not Found.



In [13]:
model = YOLO("yolov9c.pt")

In [15]:
video_path = "video2.avi"
output_video_path = "output_tracked.avi"
output_dir = "person_images"

# Function to create a folder if it doesn't exist
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder(output_dir)

# Load YOLOv9 model on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
yolo_model = model  # Load YOLOv9 model (ensure correct model file is used)
yolo_model.to(device)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

# Dictionary to store mapping of track IDs to folder names
track_id_to_folder = {}

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection on the original frame
    results = yolo_model(frame)

    detections = []  # List of detected objects

    for result in results:
        for box in result.boxes:
            cls = int(box.cls)
            if cls == 0:  # Class 0 is 'person'
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
                conf = float(box.conf[0])  # Get confidence score
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, None))  # Format for DeepSORT

    # Update tracker with detections
    tracked_objects = tracker.update_tracks(detections, frame=frame)

    for track in tracked_objects:
        if not track.is_confirmed():
            continue

        x1, y1, w, h = map(int, track.to_tlwh())
        x2, y2 = x1 + w, y1 + h
        person_id = track.track_id

        # Assign a folder name to each unique person_id
        if person_id not in track_id_to_folder:
            track_id_to_folder[person_id] = f"person_{person_id}"
        person_folder = os.path.join(output_dir, track_id_to_folder[person_id])
        create_folder(person_folder)

        # Draw bounding box and ID label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {person_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save cropped person image (ensure valid coordinates)
        if 0 <= y1 < y2 < frame.shape[0] and 0 <= x1 < x2 < frame.shape[1]:
            person_crop = frame[y1:y2, x1:x2]
            person_img_path = os.path.join(person_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(person_img_path, person_crop)
        else:
            print(f"Skipping invalid bounding box for person ID {person_id}: ({x1}, {y1}, {x2}, {y2})")

    # Write the frame to output video
    out.write(frame)
    frame_count += 1

cap.release()
out.release()
print("Processing complete. Tracked video saved as output_tracked.avi and images saved in respective folders.")

Streaming output truncated to the last 5000 lines.

0: 384x640 8 persons, 1 car, 1 stop sign, 42.7ms
Speed: 2.3ms preprocess, 42.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Skipping invalid bounding box for person ID 155: (-4, 323, 97, 696)

0: 384x640 8 persons, 2 cars, 1 bus, 43.0ms
Speed: 2.3ms preprocess, 43.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Skipping invalid bounding box for person ID 155: (-3, 329, 94, 693)

0: 384x640 8 persons, 2 cars, 1 bus, 40.2ms
Speed: 4.0ms preprocess, 40.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
Skipping invalid bounding box for person ID 155: (-3, 336, 91, 691)

0: 384x640 8 persons, 2 cars, 1 bus, 38.6ms
Speed: 2.4ms preprocess, 38.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Skipping invalid bounding box for person ID 155: (-3, 342, 89, 691)

0: 384x640 8 persons, 2 cars, 1 backpack, 32.2ms
Speed: 3.0ms preprocess, 32.2ms inference, 1.6ms postp

In [16]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
# prompt: download person_images folder as zip folder on drive

import shutil
import os
from google.colab import drive

drive.mount('/content/drive')

def zip_folder(folder_path, zip_path):
    try:
        shutil.make_archive(zip_path, 'zip', folder_path)
        print(f"Successfully zipped '{folder_path}' to '{zip_path}.zip'")
    except Exception as e:
        print(f"Error zipping folder: {e}")

# Specify the folder to zip and the desired zip file path in Google Drive
folder_to_zip = "/content/person_images"
zip_file_path = "/content/drive/My Drive/person_images"  # Change 'My Drive' if needed

# Create the zip file
zip_folder(folder_to_zip, zip_file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully zipped '/content/person_images' to '/content/drive/My Drive/person_images.zip'
